## Build a Dashboard Application with Plotly Dash
In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

- TASK 1: Add a Launch Site Drop-down Input Component
- TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
- TASK 3: Add a Range Slider to Select Payload
- TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

#### Your completed dashboard application should look like the following screenshot:
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/dash_completed.PNG">

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest

In [ ]:
!pip install wget

In [1]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import wget
import pandas as pd

In [2]:
spacex_csv_file = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

100% [................................................................................] 2476 / 2476

In [3]:
spacex_df = pd.read_csv(spacex_csv_file, index_col=0)
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

In [5]:
min_value = int(spacex_df['Payload Mass (kg)'].min())
max_value = int(spacex_df['Payload Mass (kg)'].max())

print(min_value," ", max_value)

0   9600


In [6]:
Site_options = ['All Sites'] + list(spacex_df['Launch Site'].unique())
print(Site_options)

app.layout = html.Div(children=[html.H1('Launch Performance Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                
                                html.Div([dcc.Dropdown(id='site-dropdown', multi=True, searchable = True,
                                                       options=[{'label': x, 'value': x} for x in Site_options],
                                                       value=[Site_options[0]])]),
                                
                                html.Div([dcc.Graph(id='success-pie-chart', figure={})]),
                                
                                html.P("Payload range (Kg):"),
                                html.Div([dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=500, 
                                                         marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'}, 
                                                         value=[min_value, max_value])]),
                                
                                html.Div(dcc.Graph(id='success-payload-scatter-chart', figure={})),
                               
                               ])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'),
             )


def get_pie_chart(entered_site, min_max_kg):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_max_kg[0]) & (spacex_df['Payload Mass (kg)'] <= min_max_kg[1])]
    if 'All Sites' in entered_site:
        fig = px.pie(data_frame=filtered_df, values='class', names='Launch Site', title='Total success launch by All Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        data = filtered_df[filtered_df['Launch Site'].isin(entered_site)]
        data = data.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(data_frame=data, values='class count', names='class', 
                     title=f'Total success launch for site {entered_site}')
        return fig
    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"),  
             )

def get_scatter_chart(entered_site, min_max_kg):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_max_kg[0]) & (spacex_df['Payload Mass (kg)'] <= min_max_kg[1])]
    if 'All Sites' in entered_site:
        fig2 = px.scatter(data_frame=filtered_df,x='Payload Mass (kg)',y='class',hover_data=['Launch Site'],color='Booster Version Category')
        return fig2
    else:
        data = filtered_df[filtered_df['Launch Site'].isin(entered_site)]
        fig2 = px.scatter(data_frame=data,x='Payload Mass (kg)',y='class',hover_data=['Launch Site'],color='Booster Version Category')
        return fig2
        
    
        


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

['All Sites', 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
#data_f = spacex_df[spacex_df['Launch Site'].isin(['CCAFS LC-40','CCAFS SLC-40'])]
#data_f

In [ ]:
#data_f = data_f[(data_f['Payload Mass (kg)'] >= 100) & (data_f['Payload Mass (kg)'] <= 4000)]
#data_f

In [ ]:
#data_f = data_f.groupby(['Launch Site','class']).size().reset_index(name='class count')
#data_f